In [4]:
%cd /content/drive/MyDrive/data_science_project/pytorch_tutorial/2_object_detection

/content/drive/MyDrive/data_science_project/pytorch_tutorial/2_object_detection


In [1]:
# 패키지 import
import os.path as osp
import random
import time

import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.utils.data as data

In [2]:
# 난수 시드 설정
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("사용 중인 장치:", device)

사용 중인 장치: cuda:0


# Dataset, DataLoader 작성

In [5]:
from utils.ssd_model import make_datapath_list, VOCDataset, DataTransform, Anno_xml2list, od_collate_fn

# 파일 경로 리스트를 취득
rootpath = "./data/VOCdevkit/VOC2012/"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(
    rootpath)

# Dataset 작성
voc_classes = ['aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor']
color_mean = (104, 117, 123)  # (BGR) 색의 평균값
input_size = 300  # 화상의 input 크기를 300×300으로 설정

train_dataset = VOCDataset(train_img_list, train_anno_list, phase="train", transform=DataTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))

val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DataTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))


# DataLoader를 작성
batch_size = 32

train_dataloader = data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, collate_fn=od_collate_fn)

val_dataloader = data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, collate_fn=od_collate_fn)

# 사전 오브젝트로 정리
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}


# 네트워크 모델 작성

In [6]:
from utils.ssd_model import SSD

# SSD300 설정
ssd_cfg = {
    'num_classes': 21,  # 배경 클래스를 포함한 총 클래스 수
    'input_size': 300,  # 화상의 입력 크기
    'bbox_aspect_num': [4, 6, 6, 6, 4, 4],  # 출력할 DBox의 화면비의 종류
    'feature_maps': [38, 19, 10, 5, 3, 1],  # 각 source의 화상 크기
    'steps': [8, 16, 32, 64, 100, 300],
    'min_sizes': [30, 60, 111, 162, 213, 264],  # DBOX의 크기(최소)
    'max_sizes': [60, 111, 162, 213, 264, 315],  # DBOX의 크기(최대)
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
}

# SSD 네트워크 모델
net = SSD(phase="train", cfg=ssd_cfg)

# SSD의 초기 가중치를 설정
# ssd의 vgg 부분에 가중치를 로드한다
vgg_weights = torch.load('./weights/vgg16_reducedfc.pth')
net.vgg.load_state_dict(vgg_weights)

# ssd의 기타 네트워크의 가중치는 He의 초기치로 초기화
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight.data)
        if m.bias is not None:  # 바이어스 항이 있는 경우
            nn.init.constant_(m.bias, 0.0)


# He의 초기치를 적용
net.extras.apply(weights_init)
net.loc.apply(weights_init)
net.conf.apply(weights_init)

# GPU를 사용할 수 있는지 확인
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("사용 중인 장치:", device)

print('네트워크 설정 완료: 학습된 가중치를 로드했습니다')


사용 중인 장치: cuda:0
네트워크 설정 완료: 학습된 가중치를 로드했습니다


# 손실함수 및 최적화 기법 설정

In [7]:
from utils.ssd_model import MultiBoxLoss

# 손실함수의 설정
criterion = MultiBoxLoss(jaccard_thresh=0.5, neg_pos=3, device=device)

# 최적화 기법의 설정
optimizer = optim.SGD(net.parameters(), lr=1e-3,
                      momentum=0.9, weight_decay=5e-4)


# 학습 및 검증을 실시

In [25]:
# 모델을 학습시키는 함수 작성
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPU를 사용할 수 있는지 확인
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("사용 중인 장치:", device)

    # 네트워크를 GPU로
    net.to(device)

    # 네트워크가 어느 정도 고정되면, 고속화시킨다
    torch.backends.cudnn.benchmark = True

    # 반복자의 카운터 설정
    iteration = 1
    epoch_train_loss = 0.0  # epoch의 손실합
    epoch_val_loss = 0.0  # epoch의 손실합
    logs = []

    # epoch 루프
    for epoch in range(num_epochs+1):

        # 시작 시간을 저장
        t_epoch_start = time.time()
        t_iter_start = time.time()

        print('-------------')
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # epoch별 훈련 및 검증을 루프
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # 모델을 훈련모드로
                print('(train)')
            else:
                if((epoch+1) % 10 == 0):
                    net.eval()   # 모델을 검증모드로
                    print('-------------')
                    print('(val)')
                else:
                    # 검증은 10번에 1번만 실시
                    continue

            # 데이터 로더에서 minibatch씩 꺼내 루프
            for images, targets in dataloaders_dict[phase]:

                # GPU를 사용할 수 있으면, GPU에 데이터를 보낸다
                images = images.to(device)
                targets = [ann.to(device)
                           for ann in targets]  # 리스트의 각 요소의 텐서를 GPU로

                # optimizer를 초기화
                optimizer.zero_grad()

                # 순전파(forward) 계산
                with torch.set_grad_enabled(phase == 'train'): # 안에가 True값일 때만 grad가 저장되어 역전파 수행가능
                    # 순전파(forward) 계산
                    outputs = net(images)

                    # 손실 계산
                    loss_l, loss_c = criterion(outputs, targets)
                    loss = loss_l + loss_c

                    # 훈련시에는 역전파(Backpropagation)
                    if phase == 'train':
                        loss.backward()  # 경사 계산

                        # 경사가 너무 커지면 계산이 불안정해지므로, clip에서 최대라도 경사 2.0에 고정
                        nn.utils.clip_grad_value_(
                            net.parameters(), clip_value=2.0)

                        optimizer.step()  # 파라미터 갱신

                        if (iteration % 10 == 0):  # 10iter에 한 번, loss를 표시
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            print('반복 {} || Loss: {:.4f} || 10iter: {:.4f} sec.'.format(
                                iteration, loss.item(), duration))
                            t_iter_start = time.time()

                        epoch_train_loss += loss.item()
                        iteration += 1

                    # 검증시
                    else:
                        epoch_val_loss += loss.item()

        # epoch의 phase 당 loss와 정답률
        t_epoch_finish = time.time()
        print('-------------')
        print('epoch {} || Epoch_TRAIN_Loss:{:.4f} ||Epoch_VAL_Loss:{:.4f}'.format(
            epoch+1, epoch_train_loss, epoch_val_loss))
        print('timer:  {:.4f} sec.'.format(t_epoch_finish - t_epoch_start))
        t_epoch_start = time.time()

        # 로그를 저장
        log_epoch = {'epoch': epoch+1,
                     'train_loss': epoch_train_loss, 'val_loss': epoch_val_loss}
        logs.append(log_epoch)
        df = pd.DataFrame(logs)
        df.to_csv("log_output.csv")

        epoch_train_loss = 0.0  # epoch의 손실합
        epoch_val_loss = 0.0  # epoch의 손실합

        # 네트워크를 저장한다
        if ((epoch+1) % 10 == 0):
            torch.save(net.state_dict(), 'weights/ssd300_' +
                       str(epoch+1) + '.pth')


In [26]:
# 학습 및 검증 실시
num_epochs= 50  
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

사용 중인 장치: cuda:0
-------------
Epoch 1/50
-------------
(train)


/content/drive/MyDrive/data_science_project/pytorch_tutorial/2_object_detection/utils/data_augumentation.py:254: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mode = random.choice(self.sample_options)


반복 10 || Loss: 16.5973 || 10iter: 352.0681 sec.
반복 20 || Loss: 12.9303 || 10iter: 164.0976 sec.
반복 30 || Loss: 9.8697 || 10iter: 168.8868 sec.
반복 40 || Loss: 10.3058 || 10iter: 168.3095 sec.
반복 50 || Loss: 8.5361 || 10iter: 164.5017 sec.
반복 60 || Loss: 8.2503 || 10iter: 167.5651 sec.
반복 70 || Loss: 8.3663 || 10iter: 166.8078 sec.
반복 80 || Loss: 7.5241 || 10iter: 170.0912 sec.
반복 90 || Loss: 8.0328 || 10iter: 163.6594 sec.
반복 100 || Loss: 7.7220 || 10iter: 166.6572 sec.
반복 110 || Loss: 7.4840 || 10iter: 164.5277 sec.
반복 120 || Loss: 7.1690 || 10iter: 166.8729 sec.
반복 130 || Loss: 7.6408 || 10iter: 166.2712 sec.
반복 140 || Loss: 7.3959 || 10iter: 164.9632 sec.
반복 150 || Loss: 7.1095 || 10iter: 159.5543 sec.
반복 160 || Loss: 7.5790 || 10iter: 157.0195 sec.
반복 170 || Loss: 7.2999 || 10iter: 158.9843 sec.
-------------
epoch 1 || Epoch_TRAIN_Loss:1629.3838 ||Epoch_VAL_Loss:0.0000
timer:  3136.1675 sec.
-------------
Epoch 2/50
-------------
(train)
반복 180 || Loss: 7.4900 || 10iter: 1.1483 sec

# scratch notes

In [9]:
torch.tensor([2]).item()

2

In [10]:
#torch.tensor([2, 3]).item()

ValueError: ignored

In [11]:
torch.tensor([2, 3]).data

tensor([2, 3])

In [12]:
torch.tensor([2, 3])

tensor([2, 3])

In [18]:
torch.nn.Parameter(torch.Tensor(512))

Parameter containing:
tensor([ 2.1077e-28,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         1.3769e+36,  4.5867e-41, -3.5925e+25, -5.2701e-34,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  4.6264e+35,
         4.5867e-41,  3.8736e+07, -6.1312e-31,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e

In [24]:
torch.nn.Parameter(torch.Tensor(512)).data

tensor([ 2.1097e-28,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  4.5515e+36,  4.5867e-41, -4.6488e-25,
         1.7652e+38,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  4.4347e+36,
         4.5867e-41, -8.3750e-36,  1.0440e-04,  0.0

In [19]:
torch.Tensor([512])

tensor([512.])

In [20]:
torch.tensor([512])

tensor([512])

In [22]:
torch.Tensor((512))

tensor([ 2.1092e-28,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  8.4139e+12,  4.5867e-41,
        -8.6266e-20,  1.7504e-31, -4.5949e+24,  4.5866e-41, -8.5000e+18,
        -3.1829e-17, -4.5626e+24,  4.5866e-41,  9.7343e+30,  3.8141e-33,
         1.7258e+13,  4.5867e-41,  3.1710e+21,  9.6611e-12, -4.6050e+24,
         4.5866e-41, -8.5683e-08, -2.7439e-08, -4.5424e+24,  4.5866e-41,
         1.0281e+02, -1.8788e+24, -4.5622e+24,  4.5866e-41,  2.7101e+22,
         2.0849e+10, -4.6736e+24,  4.5866e-41, -1.5138e+06, -2.3972e+07,
         4.0914e+14,  4.5867e-41, -2.3733e-17,  8.5374e+11, -4.5424e+24,
         4.5866e-41, -1.0294e+11,  1.4773e+19, -4.5938e+24,  4.5866e-41,
        -1.0608e+01, -1.0612e-08, -4.5928e+24,  4.5866e-41,  2.5240e-07,
        -1.8433e+07,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  8.4051e+12,
         4.5867e-41,  6.0169e+32,  7.7925e-03,  0.0

In [23]:
torch.tensor((512))

tensor(512)